In [1]:
import pytz
from astropy.io import ascii
import astropy.units as u
from astropy.coordinates import EarthLocation, Angle
from astroplan import Observer
from tzwhere import tzwhere
tz = tzwhere.tzwhere()
from astropy.time import Time

table = ascii.read('data.csv')

relative_times = []

relative_civil = []
relative_nautical = []
relative_astronomical = []

for i in range(len(table)): 
    lat = Angle(table['Latitude'][i], u.deg)
    lon = Angle(table['Longitude'][i], u.deg)
    
    location = EarthLocation.from_geodetic(lon, lat)
    
    timezone = tz.tzNameAt(lat.deg, lon.deg)
    
    if timezone is not None:
        obs = Observer(location=location, timezone=pytz.timezone(timezone))
        
        if (table['RecordingYear'].data[i] is not None and 
            table['RecordingDay'].data[i] is not None and 
            table['RecordingTime'].data[i] is not None): 
            
            datetime = list(map(str, [table['RecordingYear'].data[i],  
                                      table['RecordingMonth'].data[i],
                                      table['RecordingDay'].data[i], 
                                      table['RecordingTime'].data[i]]))
            try: 
                local_time = Time("{0}-{1}-{2} {3}".format(*datetime)).datetime
                localized_time = pytz.timezone(timezone).localize(local_time)
                astropy_time = Time(localized_time)
                
                sunrise = obs.sun_rise_time(astropy_time, which='nearest')
                astro = obs.twilight_morning_astronomical(astropy_time, which='nearest')
                civil = obs.twilight_morning_civil(astropy_time, which='nearest')
                nautical = obs.twilight_morning_nautical(astropy_time, which='nearest')
                
                if sunrise < astropy_time: 
                    relative_times.append('after sunrise')
                else: 
                    relative_times.append('before sunrise')

                if civil < astropy_time:
                    relative_civil.append('after civil')
                else:
                    relative_civil.append('before civil')
                if nautical < astropy_time:
                    relative_nautical.append('after nautical')
                else: 
                    relative_nautical.append('before nautical')
                if astro < astropy_time: 
                    relative_astronomical.append('after astronomical')
                else: 
                    relative_astronomical.append('before astronomical')
    
            except ValueError: 
                relative_times.append('--')
                relative_astronomical.append('--')
                relative_civil.append('--')
                relative_nautical.append('--')
        else: 
            relative_times.append('--')
            relative_astronomical.append('--')
            relative_civil.append('--')
            relative_nautical.append('--')
    else: 
        relative_times.append('--')
        relative_astronomical.append('--')
        relative_civil.append('--')
        relative_nautical.append('--')

In [2]:
# with open('relative_times.csv', 'w') as w:
#     w.write('\n'.join(relative_times))

In [3]:
from astropy.io import ascii
from astropy.table import Table

table = Table([relative_times, relative_civil, relative_nautical, relative_astronomical])

ascii.write(table, format='csv', output='relative_times.csv')